In [12]:
from keras.models import Sequential
from keras import layers
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint

import tensorflow as tf
import pandas as pd
import numpy as np
import sagemaker
import os
from sagemaker.tensorflow import TensorFlow
import time
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


from matplotlib import pyplot as plt
import os
from collections import Counter
from tqdm import tqdm

In [13]:
tqdm.pandas()

In [2]:
input_path = 'data/input/'

In [3]:
all_train_data = pd.read_csv(input_path + 'train_data_prepped.csv').fillna('')
all_test_data = pd.read_csv(input_path + 'test_data_prepped.csv').fillna('')

In [9]:
all_train_data.head()

,id,text,cat,text_no_punc,text_no_numerals,text_no_sw,text_porter_stemmed,text_lancaster_stemmed,text_lemmatized,target
0,0,anyway im getting of for a while,train,anyway im getting of for a while,anyway im getting of for a while,anyway im getting,anyway im get,anyway im get,anyway im getting,2
1,1,"my red, apache isn't feelin too well this morn...",train,my red apache isnt feelin too well this mornin...,my red apache isnt feelin too well this mornin...,red apache feelin well morning httpmypictmen,red apach feelin well morn httpmypictmen,red apach feelin wel morn httpmypictmen,red apache feelin well morning httpmypictmen,0
2,2,@danyelljoy you should be its great. friday w...,train,danyelljoy you should be its great friday wil...,danyelljoy you should be its great friday wil...,danyelljoy great friday great tooooooo,danyelljoy great friday great tooooooo,danyelljoy gre friday gre tooooooo,danyelljoy great friday great tooooooo,2
3,3,its 11:30pm and i dont wanna sleep; so i debat...,train,its 1130pm and i dont wanna sleep so i debated...,its pm and i dont wanna sleep so i debated wit...,pm wanna sleep debated end decided perfect tim...,pm wanna sleep debat end decid perfect time ba...,pm wann sleep deb end decid perfect tim bake! kid,pm wanna sleep debated end decided perfect tim...,2
4,4,why does twitter eat my dm's? not happy,train,why does twitter eat my dms? not happy,why does twitter eat my dms? not happy,twitter eat dms? happy,twitter eat dms? happi,twit eat dms? happy,twitter eat dms? happy,0


In [4]:
train_target = pd.read_csv(input_path + 'train_results.csv')
train_target['target'].value_counts()


positive    520436
negative    519803
neutral         84
Name: target, dtype: int64

In [5]:
train_target['target'] = [0 if t == 'negative' else 2 if t == 'positive' else 2 for t in train_target['target'].values]

In [6]:
all_train_data['target'] = train_target['target']

## get word count from corpus

In [8]:
vocab = Counter()

In [16]:
for i, row in tqdm(all_train_data.iterrows(), total=all_train_data.shape[0]):
    text = row['text_no_sw'].split()
    
    
    vocab.update(text)
    
    
    
    

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1040323/1040323 [01:21<00:00, 12702.57it/s]


In [17]:
print(vocab.most_common(50))

[('im', 152152), ('good', 73009), ('get', 70120), ('like', 66148), ('day', 65143), ('go', 60098), ('going', 54166), ('love', 53584), ('cant', 53487), ('got', 52151), ('work', 51348), ('today', 47895), ('back', 45870), ('time', 44625), ('lol', 44008), ('u', 43114), ('one', 42227), ('know', 41327), ('really', 41135), ('see', 38982), ('well', 36800), ('still', 36714), ('new', 35986), ('want', 35761), ('amp', 34913), ('think', 34660), ('oh', 32033), ('night', 31996), ('home', 31774), ('miss', 30772), ('need', 30444), ('last', 30186), ('thanks', 29757), ('much', 28906), ('hope', 28262), ('ill', 27253), ('thats', 26072), ('great', 25822), ('feel', 25649), ('morning', 24800), ('twitter', 24437), ('tomorrow', 24344), ('haha', 24018), ('wish', 23605), ('sad', 23147), ('would', 23014), ('bad', 22214), ('happy', 21527), ('sleep', 21287), ('make', 21117)]
